# Import

In [45]:
import pandas as pd
import numpy as np
from gensim.models import LdaModel, TfidfModel, CoherenceModel
from gensim.corpora import Dictionary
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from gensim.utils import simple_preprocess
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, accuracy_score, f1_score


# Data

In [53]:
df = pd.read_csv('./datasets/annotated_data_lda.csv', error_bad_lines=False)
df

C:\Users\researcher\AppData\Local\Temp\ipykernel_7888\3019909967.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('./datasets/annotated_data_lda.csv', error_bad_lines=False)


,text,Order,topic,topic_number,five_emo,three_emo_0n1n2p,three_emo_0n1n2p2,preprocessed_reviews_lda,wordtoken_lda
0,"accessibility options are very diverse, maybe...",18,Accessibility options,2,4.0,2,0,accessibility option diverse maybe colourblind...,"['accessibility', 'option', 'diverse', 'maybe'..."
1,the colour schemes for stars and economies ar...,313,Accessibility options,2,1.0,0,2,colour scheme star economy disability friendly,"['colour', 'scheme', 'economy', 'disability', ..."
2,there is still no left-handed support,371,Accessibility options,2,1.0,0,2,still no left handed support,"['still', 'no', 'left', 'handed', 'support']"
3,"Also, every time the state of your suit is upd...",566,Accessibility options,2,4.0,2,0,also every time state suit update suit voice t...,"['also', 'every', 'time', 'state', 'suit', 'up..."
4,As a left handed I can't play this game the wa...,657,Accessibility options,2,1.0,0,2,left hand play way every games,"['left', 'hand', 'play', 'way', 'every']"
...,...,...,...,...,...,...,...,...,...
3106,seeing it on my Wishlist for way too long.,2137,Wishlist and Notifications,30,3.0,1,1,see wishlist way long,"['see', 'wishlist', 'way', 'long']"
3107,this game has been on my Wishlist since the pr...,2714,Wishlist and Notifications,30,4.0,2,0,wishlist since pre order come finally go bough...,"['wishlist', 'pre', 'order', 'finally', 'go', ..."
3108,this game should be the next on your Wishlist.,2766,Wishlist and Notifications,30,5.0,2,0,next wishlist,"['next', 'wishlist']"
3109,This game was in my Wishlist for a while,2771,Wishlist and Notifications,30,3.0,1,1,wishlist,['wishlist']


In [54]:
data = df.wordtoken_lda.values.tolist()
data[:3]

["['accessibility', 'option', 'diverse', 'maybe', 'colourblind', 'mode', 'would', 'awesome', 'excuse', 'already', 'know']",
 "['colour', 'scheme', 'economy', 'disability', 'friendly']",
 "['still', 'no', 'left', 'handed', 'support']"]

In [55]:
tokenized_docs = [simple_preprocess(doc) for doc in data]

In [56]:
# Create a dictionary representation of the documents
dictionary = Dictionary(tokenized_docs)

# Filter out extreme values
dictionary.filter_extremes(no_below=1, no_above=0.9)

# Create the Bag-of-Words (BoW) representation of the documents
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

# Train LDA Model with BoW

In [57]:
num_topics = 62
lda_model_bow = LdaModel(corpus=bow_corpus, id2word=dictionary, num_topics=num_topics, random_state=42)

In [58]:
# Calculate Coherence Score for BoW
coherence_model_bow = CoherenceModel(model=lda_model_bow, texts=tokenized_docs, dictionary=dictionary, coherence='c_v')
coherence_score_bow = coherence_model_bow.get_coherence()
print(f"Coherence Score (BoW): {coherence_score_bow}")

Coherence Score (BoW): 0.42541493453250434


# Use TF-IDF

In [59]:
# Create the TF-IDF model
tfidf = TfidfModel(bow_corpus)

# Create the TF-IDF representation of the documents
tfidf_corpus = tfidf[bow_corpus]

# Train LDA model with TF-IDF
lda_model_tfidf = LdaModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=num_topics, random_state=42)

# Calculate Coherence Score for TF-IDF
coherence_model_tfidf = CoherenceModel(model=lda_model_tfidf, texts=tokenized_docs, dictionary=dictionary, coherence='c_v')
coherence_score_tfidf = coherence_model_tfidf.get_coherence()
print(f"Coherence Score (TF-IDF): {coherence_score_tfidf}")


Coherence Score (TF-IDF): 0.4496525497769263


# Get Topic Distributions for Documents

In [60]:
# Get topic distributions for documents
def get_topic_distribution(model, corpus):
    topic_distributions = []
    for doc in corpus:
        topic_distribution = model.get_document_topics(doc, minimum_probability=0.0)
        topic_distribution = [prob for _, prob in topic_distribution]
        topic_distributions.append(topic_distribution)
    return np.array(topic_distributions)

topic_distributions_bow = get_topic_distribution(lda_model_bow, bow_corpus)
topic_distributions_tfidf = get_topic_distribution(lda_model_tfidf, tfidf_corpus)

# Train a Supervised Model

In [61]:
responses = df['topic_number']
responses

0        2
1        2
2        2
3        2
4        2
        ..
3106    30
3107    30
3108    30
3109    30
3110    30
Name: topic_number, Length: 3111, dtype: int64

In [62]:
# Train the logistic regression model for BoW
classifier_bow = LogisticRegression(max_iter=1000)
classifier_bow.fit(topic_distributions_bow, responses)

LogisticRegression(max_iter=1000)

In [63]:
# Predict the responses for BoW
predicted_responses_bow = classifier_bow.predict(topic_distributions_bow)

# Evaluate the model for BoW
precision_bow = precision_score(responses, predicted_responses_bow, average='macro')
accuracy_bow = accuracy_score(responses, predicted_responses_bow)
f1_bow = f1_score(responses, predicted_responses_bow, average='macro')

print(f"Precision (BoW): {precision_bow}")
print(f"Accuracy (BoW): {accuracy_bow}")
print(f"F1 Score (BoW): {f1_bow}")

Precision (BoW): 0.053482936911589805
Accuracy (BoW): 0.13436194149791064
F1 Score (BoW): 0.03506227525491915


D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
# Train the logistic regression model for TF-IDF
classifier_tfidf = LogisticRegression(max_iter=1000)
classifier_tfidf.fit(topic_distributions_tfidf, responses)

# Predict the responses for TF-IDF
predicted_responses_tfidf = classifier_tfidf.predict(topic_distributions_tfidf)

# Evaluate the model for TF-IDF
precision_tfidf = precision_score(responses, predicted_responses_tfidf, average='macro')
accuracy_tfidf = accuracy_score(responses, predicted_responses_tfidf)
f1_tfidf = f1_score(responses, predicted_responses_tfidf, average='macro')

print(f"Precision (TF-IDF): {precision_tfidf}")
print(f"Accuracy (TF-IDF): {accuracy_tfidf}")
print(f"F1 Score (TF-IDF): {f1_tfidf}")

Precision (TF-IDF): 0.06426133320114542
Accuracy (TF-IDF): 0.13468338154934104
F1 Score (TF-IDF): 0.02970868645852679


D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [66]:
# Display the top 10 words for each topic in BoW LDA model
print("\nTop 10 words for each topic (BoW):")
for idx, topic in lda_model_bow.print_topics(num_topics=num_topics, num_words=10):
    print(f"Topic {idx + 1}: {topic}")



Top 10 words for each topic (BoW):
Topic 1: 0.036*"planet" + 0.026*"console" + 0.018*"enjoyment" + 0.016*"dlcs" + 0.016*"add" + 0.015*"screenshots" + 0.014*"real" + 0.012*"number" + 0.012*"platform" + 0.011*"ability"
Topic 2: 0.039*"interface" + 0.028*"buy" + 0.021*"either" + 0.017*"plot" + 0.016*"easy" + 0.015*"play" + 0.015*"way" + 0.014*"run" + 0.014*"button" + 0.013*"think"
Topic 3: 0.028*"story" + 0.023*"navigate" + 0.017*"like" + 0.017*"could" + 0.015*"fun" + 0.013*"thing" + 0.012*"avoid" + 0.011*"want" + 0.011*"read" + 0.011*"entire"
Topic 4: 0.032*"event" + 0.029*"base" + 0.019*"time" + 0.016*"much" + 0.016*"beautiful" + 0.015*"love" + 0.015*"launch" + 0.014*"year" + 0.013*"visually" + 0.013*"even"
Topic 5: 0.047*"refund" + 0.029*"hour" + 0.029*"story" + 0.027*"interface" + 0.020*"user" + 0.019*"great" + 0.016*"maybe" + 0.015*"get" + 0.015*"need" + 0.013*"somewhere"
Topic 6: 0.033*"get" + 0.031*"student" + 0.028*"play" + 0.024*"virtual" + 0.023*"repetitive" + 0.023*"vision" + 

In [67]:
# Display the top 10 words for each topic in TF-IDF LDA model
print("\nTop 10 words for each topic (TF-IDF):")
for idx, topic in lda_model_tfidf.print_topics(num_topics=num_topics, num_words=10):
    print(f"Topic {idx + 1}: {topic}")


Top 10 words for each topic (TF-IDF):
Topic 1: 0.114*"content" + 0.068*"downloadable" + 0.050*"add" + 0.035*"everyone" + 0.033*"although" + 0.028*"weapon" + 0.024*"ago" + 0.023*"would" + 0.017*"story" + 0.013*"hello"
Topic 2: 0.081*"computer" + 0.046*"never" + 0.030*"headset" + 0.027*"another" + 0.025*"craft" + 0.020*"system" + 0.017*"somewhat" + 0.012*"play" + 0.011*"foot" + 0.011*"google"
Topic 3: 0.068*"planet" + 0.030*"small" + 0.029*"ship" + 0.027*"swat" + 0.022*"massive" + 0.022*"would" + 0.018*"idea" + 0.018*"simply" + 0.015*"land" + 0.012*"get"
Topic 4: 0.055*"base" + 0.036*"beautiful" + 0.030*"choice" + 0.020*"random" + 0.019*"event" + 0.014*"target" + 0.012*"mind" + 0.012*"pointless" + 0.011*"satisfy" + 0.011*"many"
Topic 5: 0.039*"fully" + 0.029*"somewhere" + 0.007*"suddenly" + 0.004*"gameplay" + 0.004*"would" + 0.002*"tucked" + 0.002*"single" + 0.001*"away" + 0.001*"bigfoot" + 0.001*"tribe"
Topic 6: 0.066*"good" + 0.039*"virtual" + 0.037*"reality" + 0.035*"could" + 0.030*"

In [68]:
# Reflect the topic identification results in the original documents
def reflect_topics_in_documents(model, corpus, documents):
    topic_documents = []
    for i, doc in enumerate(corpus):
        topic_distribution = model.get_document_topics(doc, minimum_probability=0.0)
        dominant_topic = max(topic_distribution, key=lambda x: x[1])[0]
        topic_documents.append((documents[i], dominant_topic))
    return topic_documents


In [70]:
# Reflect topics in documents for BoW
topic_documents_bow = reflect_topics_in_documents(lda_model_bow, bow_corpus, data)
print("\nDocuments and their dominant topics (BoW):")
for doc, topic in topic_documents_bow:
    print(f"Document: {doc}\nDominant Topic: {topic + 1}\n")



Documents and their dominant topics (BoW):
Document: ['accessibility', 'option', 'diverse', 'maybe', 'colourblind', 'mode', 'would', 'awesome', 'excuse', 'already', 'know']
Dominant Topic: 32

Document: ['colour', 'scheme', 'economy', 'disability', 'friendly']
Dominant Topic: 52

Document: ['still', 'no', 'left', 'handed', 'support']
Dominant Topic: 9

Document: ['also', 'every', 'time', 'state', 'suit', 'update', 'suit', 'voice', 'tell', 'status', 'nice', 'first', 'time', 'useful', 'visually', 'impaired']
Dominant Topic: 4

Document: ['left', 'hand', 'play', 'way', 'every']
Dominant Topic: 2

Document: ['play', 'anymore', 'full', 'screen', 'white', 'flash', 'transitory', 'action', 'engaging', 'disengaging', 'pulse', 'engine', 'finish', 'load', 'etc', 'turn', 'eye', 'soup', 'give', 'migraine', 'within', 'minute']
Dominant Topic: 57

Document: ['no', 'option', 'support', 'inclusion', 'subtitle', 'hard', 'hearing', 'person', 'impossible', 'play']
Dominant Topic: 6

Document: ['hold', 'c

In [72]:
# Reflect topics in documents for TF-IDF
topic_documents_tfidf = reflect_topics_in_documents(lda_model_tfidf, tfidf_corpus, data)
print("\nDocuments and their dominant topics (TF-IDF):")
for doc, topic in topic_documents_tfidf:
    print(f"Document: {doc}\nDominant Topic: {topic + 1}\n")


Documents and their dominant topics (TF-IDF):
Document: ['accessibility', 'option', 'diverse', 'maybe', 'colourblind', 'mode', 'would', 'awesome', 'excuse', 'already', 'know']
Dominant Topic: 9

Document: ['colour', 'scheme', 'economy', 'disability', 'friendly']
Dominant Topic: 18

Document: ['still', 'no', 'left', 'handed', 'support']
Dominant Topic: 45

Document: ['also', 'every', 'time', 'state', 'suit', 'update', 'suit', 'voice', 'tell', 'status', 'nice', 'first', 'time', 'useful', 'visually', 'impaired']
Dominant Topic: 30

Document: ['left', 'hand', 'play', 'way', 'every']
Dominant Topic: 28

Document: ['play', 'anymore', 'full', 'screen', 'white', 'flash', 'transitory', 'action', 'engaging', 'disengaging', 'pulse', 'engine', 'finish', 'load', 'etc', 'turn', 'eye', 'soup', 'give', 'migraine', 'within', 'minute']
Dominant Topic: 13

Document: ['no', 'option', 'support', 'inclusion', 'subtitle', 'hard', 'hearing', 'person', 'impossible', 'play']
Dominant Topic: 47

Document: ['hol

In [73]:
# Function to get the most representative topic for each document
def get_most_representative_topic(model, corpus):
    topics = []
    for doc in corpus:
        topic_distribution = model.get_document_topics(doc, minimum_probability=0.0)
        most_representative_topic = max(topic_distribution, key=lambda x: x[1])[0]
        topics.append(most_representative_topic)
    return topics

# Get the most representative topic for each document
most_representative_topic_bow = get_most_representative_topic(lda_model_bow, bow_corpus)
most_representative_topic_tfidf = get_most_representative_topic(lda_model_tfidf, tfidf_corpus)


In [83]:
# Create a pandas DataFrame to store the results
df_bow = pd.DataFrame({
    'Document': data,
    'Original Category': df['topic_number'],
    'Predicted Category': predicted_responses_bow,
    'Most Representative Topic (BoW)': most_representative_topic_bow
})
df_bow

,Document,Original Category,Predicted Category,Most Representative Topic (BoW)
0,"['accessibility', 'option', 'diverse', 'maybe'...",2,51,31
1,"['colour', 'scheme', 'economy', 'disability', ...",2,51,51
2,"['still', 'no', 'left', 'handed', 'support']",2,51,8
3,"['also', 'every', 'time', 'state', 'suit', 'up...",2,40,3
4,"['left', 'hand', 'play', 'way', 'every']",2,51,1
...,...,...,...,...
3106,"['see', 'wishlist', 'way', 'long']",30,51,27
3107,"['wishlist', 'pre', 'order', 'finally', 'go', ...",30,51,15
3108,"['next', 'wishlist']",30,51,43
3109,['wishlist'],30,51,0


In [84]:
df_tfidf = pd.DataFrame({
    'Document': data,
    'Original Category': df['topic_number'],
    'Predicted Category': predicted_responses_tfidf,
    'Most Representative Topic (TF-IDF)': most_representative_topic_tfidf
})
df_tfidf

,Document,Original Category,Predicted Category,Most Representative Topic (TF-IDF)
0,"['accessibility', 'option', 'diverse', 'maybe'...",2,51,8
1,"['colour', 'scheme', 'economy', 'disability', ...",2,51,17
2,"['still', 'no', 'left', 'handed', 'support']",2,51,44
3,"['also', 'every', 'time', 'state', 'suit', 'up...",2,51,7
4,"['left', 'hand', 'play', 'way', 'every']",2,51,27
...,...,...,...,...
3106,"['see', 'wishlist', 'way', 'long']",30,51,8
3107,"['wishlist', 'pre', 'order', 'finally', 'go', ...",30,51,38
3108,"['next', 'wishlist']",30,51,54
3109,['wishlist'],30,51,0
